# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [88]:
import pandas as pd

In [89]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [90]:
df1 = df1.drop_duplicates()

In [91]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [92]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [93]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [94]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [95]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [96]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [97]:
from datetime import timedelta, datetime

In [98]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], format='%d/%m/%Y')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

In [99]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [100]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])
df1['screening_days'] = df1['screening_days'].astype(int)

In [101]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [102]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [103]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [104]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,32,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,12,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,19,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,11,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,7,The Old Oak,2023-11-16,GBR,Lucky Red Distrib.,71051.19,10467,662869.90,101942,2023-11-27,11,2023-11-26,Sunday,6.79
117,8,Comandante,2023-10-31,ITA,01 Distribution,45779.74,6800,3449848.07,509223,2023-11-27,27,2023-11-26,Sunday,6.73
118,9,Mary E Lo Spirito Di Mezzanotte,2023-11-23,ITA,Bim Distrib. S.R.L.,43954.04,6679,90295.77,14093,2023-11-27,4,2023-11-26,Sunday,6.58
119,10,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,36518.44,4499,639027.23,83583,2023-11-27,11,2023-11-26,Sunday,8.12


---

## Export as .csv

In [105]:
import os

In [106]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [107]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [108]:
import seaborn as sns
import matplotlib.pyplot as plt

In [109]:
pd.set_option('display.max_rows', 8)

In [110]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110 entries, 0 to 119
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            110 non-null    int32         
 1   title                 110 non-null    object        
 2   first_screening_date  110 non-null    object        
 3   nation                110 non-null    object        
 4   distribution          110 non-null    object        
 5   daily_takings         110 non-null    float64       
 6   daily_attendance      110 non-null    int32         
 7   total_takings         110 non-null    float64       
 8   total_attendance      110 non-null    int32         
 9   date_pulled           110 non-null    object        
 10  screening_days        110 non-null    int32         
 11  date                  110 non-null    datetime64[ns]
 12  day_of_week           110 non-null    object        
 13  avg_ticket_price      110

In [111]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,date,avg_ticket_price
count,110.00,110.00,110.00,110.00,110.00,110.00,110,110.00
mean,5.50,189540.58,26518.62,3670756.40,525011.80,19.44,2023-11-21 00:00:00,6.88
min,1.00,8904.68,1458.00,11652.78,1997.00,4.00,2023-11-16 00:00:00,4.22
25%,3.00,31897.57,4731.00,410962.41,56586.25,11.00,2023-11-18 00:00:00,6.31
50%,5.50,53180.66,7754.50,2074384.39,300455.50,15.00,2023-11-21 00:00:00,6.80
75%,8.00,206832.46,28887.75,3353017.14,494319.00,27.00,2023-11-24 00:00:00,7.41
max,10.00,1544231.00,211764.00,23908728.22,3468044.00,81.00,2023-11-26 00:00:00,10.94
std,2.89,298531.32,41336.62,5642971.33,817734.15,15.20,NaN,1.05


In [112]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    0
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [113]:
df1.nunique()

daily_rank               10
title                    17
first_screening_date     11
nation                    5
                       ... 
screening_days           11
date                     11
day_of_week               7
avg_ticket_price        110
Length: 14, dtype: int64

In [114]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
daily_rank,1.00,-0.67,-0.67,-0.45,-0.45,0.15,-0.30
daily_takings,-0.67,1.00,0.99,0.68,0.68,0.05,0.21
daily_attendance,-0.67,0.99,1.00,0.73,0.72,0.08,0.16
total_takings,-0.45,0.68,0.73,1.00,1.00,0.44,-0.07
total_attendance,-0.45,0.68,0.72,1.00,1.00,0.46,-0.10
screening_days,0.15,0.05,0.08,0.44,0.46,1.00,-0.47
avg_ticket_price,-0.30,0.21,0.16,-0.07,-0.10,-0.47,1.00


In [115]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,1.18,817831.27,119313.18,19699250.33,2850002.36,32.00,6.73
Io Capitano,8.50,15849.42,3666.00,4214800.04,732489.25,81.00,4.34
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,39.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,39.00,7.20
...,...,...,...,...,...,...,...
Dallamericaruso. Il Concerto Perduto,3.00,116131.38,10790.67,206934.63,19372.67,7.00,10.71
Dream Scenario - Hai Mai Sognato Quest'Uomo?,9.43,24143.02,3622.86,101342.57,14879.43,11.00,6.47
Mary E Lo Spirito Di Mezzanotte,9.50,38548.18,5860.50,68220.25,10738.50,4.00,6.58
La Chimera,10.00,12522.45,1942.50,19722.90,3210.50,4.00,6.38


In [116]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,1.18,817831.27,119313.18,19699250.33,2850002.36,32.00,6.73
Io Capitano,8.50,15849.42,3666.00,4214800.04,732489.25,81.00,4.34
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,39.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,39.00,7.20
...,...,...,...,...,...,...,...
Dallamericaruso. Il Concerto Perduto,3.00,116131.38,10790.67,206934.63,19372.67,7.00,10.71
Dream Scenario - Hai Mai Sognato Quest'Uomo?,9.43,24143.02,3622.86,101342.57,14879.43,11.00,6.47
Mary E Lo Spirito Di Mezzanotte,9.50,38548.18,5860.50,68220.25,10738.50,4.00,6.58
La Chimera,10.00,12522.45,1942.50,19722.90,3210.50,4.00,6.38
